# Calibration of the 9-Mythen detector at the Cristal beamline at Soleil

Mythen detectors are 1D-strip detector sold by Dectris. 
On the Cristal beamline at Soleil, 9 of them are mounted on the goniometer. 

This notebook explains how to calibrate precisely their position (including the wavelength used) as function of the goniometer position.

All input data are provided in a Nexus file wich contrains both the (approximate) energy, the goniometer positions (500 points have been measured) and the measured signal.

As pyFAI is not made for 1D data, the Mythen detector will be considered as a 1x1280 image.

We start by importing a whole bunch of modules:

In [1]:
from collections import OrderedDict
import ipympl
from matplotlib import pyplot as plt
import numpy

import h5py

from pyFAI import goniometer
from pyFAI.detectors import Detector
from pyFAI.goniometer import ExtendedTransformation, GoniometerRefinement
from pyFAI.control_points import ControlPoints
from pyFAI.geometryRefinement import GeometryRefinement
from pyFAI.gui import jupyter
from pyFAI.units import hc
from pyFAI.calibrant import get_calibrant
from pyFAI.containers import Integrate1dResult

import ipywidgets as widgets

from scipy.signal import find_peaks_cwt
from scipy.interpolate import interp1d
from scipy.optimize import bisect, minimize
from scipy.spatial import distance_matrix
import time

start_time = time.time()

/mntdirect/_scisoft/users/jupyter/jupy35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


The data file can be downoaded from:
http://www.silx.org/pub/pyFAI/gonio/LaB6_17keV_att3_tth2C_24_01_2018_19-43-20_1555.nxs

In [2]:
#Open the Nexus file and retrieve the actual positions:

h5 = h5py.File("LaB6_17keV_att3_tth2C_24_01_2018_19-43-20_1555.nxs", mode="r")
position = h5["/LaB6_17keV_att3_1555/scan_data/actuator_1_1"][:]
print("Positions: ", position[:5], "...")

Positions:  [90.00000001 89.79994445 89.5998889  89.39994445 89.19994445] ...


In [3]:
#Read all data

data = {}
ds_names = []
for idx in range(1,13):
    name = "data_%02i"%idx
    ds = h5["/LaB6_17keV_att3_1555/scan_data/"+name][:]
    print(name, ds.shape)
    if ds.shape[1]<2000:
        #Keep only the single modules
        data[name] = ds
        ds_names.append(name)

print(ds_names)


data_01 (501, 5120)
data_02 (501, 1280)
data_03 (501, 1280)
data_04 (501, 1280)
data_05 (501, 1280)
data_06 (501, 5120)
data_07 (501, 1280)
data_08 (501, 1280)
data_09 (501, 1280)
data_10 (501, 1280)
data_11 (501, 1280)
data_12 (501, 1280)
['data_02', 'data_03', 'data_04', 'data_05', 'data_07', 'data_08', 'data_09', 'data_10', 'data_11', 'data_12']


In [4]:
#Define a Mythen-detector mounted vertically:

class MythenV(Detector):
    "Verical Mythen dtrip detector from Dectris"
    aliases = ["MythenV 1280"]
    force_pixel = True
    MAX_SHAPE = (1280, 1)

    def __init__(self,pixel1=50e-6, pixel2=8e-3):
        super(MythenV, self).__init__(pixel1=pixel1, pixel2=pixel2)



In [5]:
#Define all modules as single detectors of class MythenV. 
# Each one has a mask defined from dummy-values in the dataset

modules = {}
for name, ds in data.items():
    one_module = MythenV()
    mask = ds[0]<0
    #discard the first 20 and last 20 pixels as their intensities are less reliable
    mask[:20] = True
    mask[-20:] = True
    one_module.mask = mask.reshape(-1,1)
    modules[name] = one_module

for k,v in modules.items():
    print(k, v.name)

data_12 MythenV 1280
data_11 MythenV 1280
data_10 MythenV 1280
data_04 MythenV 1280
data_07 MythenV 1280
data_02 MythenV 1280
data_03 MythenV 1280
data_08 MythenV 1280
data_05 MythenV 1280
data_09 MythenV 1280


In [6]:
# Define a peak-picking function based on the dataset-name and the frame_id:

def peak_picking(module_name, frame_id,  
                 threshold=500):
    """Peak-picking base on find_peaks_cwt from scipy plus 
    second-order tailor exapention refinement for sub-pixel resolution.
    
    The half-pixel offset is accounted here, i.e pixel #0 has its center at 0.5
    
    """
    module = modules[module_name]
    msk = module.mask.ravel()
    
    spectrum = data[module_name][frame_id]
    guess = find_peaks_cwt(spectrum, [20])
    
    valid = numpy.logical_and(numpy.logical_not(msk[guess]), 
                               spectrum[guess]>threshold)
    guess = guess[valid]
    
    #Based on maximum is f'(x) = 0 ~ f'(x0) + (x-x0)*(f''(x0))
    df = numpy.gradient(spectrum)
    d2f = numpy.gradient(df)
    bad = d2f==0
    d2f[bad] = 1e-10 #prevent devision by zero. Discared later on
    cor = df / d2f
    cor[abs(cor)>1] = 0
    cor[bad] = 0
    ref = guess - cor[guess] + 0.5 #half a pixel offset
    x = numpy.zeros_like(ref) + 0.5 #half a pixel offset
    return numpy.vstack((ref,x)).T

%timeit peak_picking(ds_names[0], 93)
print(peak_picking(ds_names[0], 93))

18.9 ms ± 761 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
[[287.06072343   0.5       ]]


In [7]:
nrj = h5["/LaB6_17keV_att3_1555/CRISTAL/Monochromator/energy"][0]
wl = hc / nrj *1e-10
print("Energy (keV): ",nrj, "\nWavelength (A): ",wl)

LaB6 = get_calibrant("LaB6")
LaB6.wavelength = wl
print(LaB6)

Energy (keV):  17.027082549190933 
Wavelength (A):  7.281587849134994e-11
LaB6 Calibrant at wavelength 7.281587849134994e-11


In [8]:
#This cell defines the transformation of coordinates for a simple goniometer mounted vertically.

trans = ExtendedTransformation(dist_expr="dist", 
                                   poni1_expr="poni1", 
                                   poni2_expr="poni2", 
                                   rot1_expr="rot1", 
                                   rot2_expr="pi*(offset+scale*angle)/180.", 
                                   rot3_expr="0.0", 
                                   wavelength_expr="hc/nrj*1e-10", 
                                   param_names=["dist", "poni1", "poni2", "rot1", "offset", "scale", "nrj"], 
                                   pos_names=["angle"], 
                                   constants={"hc": hc})

In [9]:
def get_position(idx):
    "Returns the postion of the goniometer for the given frame_id"
    return position[idx]

#Approximate offset for the module #0 at 0°
print("Approximated offset for the first module: ",get_position(36))

Approximated offset for the first module:  82.79994445106844


In [10]:
#This interactive plot lets one visualize any spectra acquired by any module

fig, ax = plt.subplots()
line = ax.plot(data[ds_names[0]][250])[0]
ligne = plt.Line2D(xdata=[640,640], ydata=[-500, 1000], figure=fig, linestyle="--", color='red', axes=ax)
ax.add_line(ligne)
ax.set_title("spectrum")
fig.show()

def update(module_id, frame_id):
   spectrum = data[ds_names[module_id]][frame_id]
   line.set_data(numpy.arange(spectrum.size), spectrum)
   ax.set_title("Module %i, Frame %i"%(module_id, frame_id))
   
   fig.canvas.draw()

    
interactive_plot = widgets.interactive(update, 
                                       module_id=(0, len(data)-1), 
                                       frame_id=(0, data[ds_names[0]].shape[0]-1))
display(interactive_plot)

/mntdirect/_scisoft/users/jupyter/jupy35/lib/python3.5/site-packages/matplotlib/__init__.py:932: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


FigureCanvasNbAgg()

interactive(children=(IntSlider(value=4, description='module_id', max=9), IntSlider(value=250, description='frame_id', max=500), Output()), _dom_classes=('widget-interact',))

In [11]:
#Work with the first module corresponding to:
name = ds_names[0]
print(name)
ds = data[name]
module = modules[name]

#Use the previous widget to select:
## the index where the beam-center is in the middle of the module
zero_pos = 36

## The frame index where the first LaB6 peak enters the right-hand side of the spectrum
peak_zero_start = 74

## The frame index where this first LaB6 leaves the spectrum or the second LaB6 peak appears:
peak_zero_end = 94

# The frames between peak_zero_start and peak_zero_end will be used to calibrate roughly the goniometer 
# and used later for finer peak extraction

data_02


In [12]:
param0 = {"dist": 0.72, 
          "poni1": 640*50e-6, 
          "poni2": 4e-3, 
          "rot1":0, 
          "offset": -get_position(zero_pos), 
          "scale":1, 
          "nrj": nrj}

#Lock enegy for now and a couple of other parameters
bounds0 = {"nrj": (nrj, nrj),
           "dist": (0.71, 0.73),
           "poni2": (4e-3, 4e-3),
           "rot1": (0,0),
           "scale":(1,1), 
          }

gonioref0 = GoniometerRefinement(param0, 
                                 get_position, 
                                 trans, 
                                 detector=module, 
                                 wavelength=wl, 
                                 bounds=bounds0
                                 )
goniometers = {name:  gonioref0} 
print(gonioref0)

GoniometerRefinement with 0 geometries labeled: .


In [13]:
# Extract the frames where only the peak zero from LaB6 is present.

for i in range(peak_zero_start, peak_zero_end):
    cp = ControlPoints(calibrant=LaB6, wavelength=wl)
    peak = peak_picking(name, i)
    if len(peak)!=1: 
        continue
    cp.append([peak[0]], ring=0)
    img = ds[i].reshape((-1,1)) #Images are vertical ... transpose the spectrum
    sg = gonioref0.new_geometry("%s_%04i"%(name,i), 
                                image=img, 
                                metadata=i, 
                                control_points=cp, 
                                calibrant=LaB6)
    sg.geometry_refinement.data = numpy.array(cp.getList())

print(gonioref0)
print("Residual error before fit:")
print(gonioref0.chi2())

GoniometerRefinement with 20 geometries labeled: data_02_0074, data_02_0075, data_02_0076, data_02_0077, data_02_0078, data_02_0079, data_02_0080, data_02_0081, data_02_0082, data_02_0083, data_02_0084, data_02_0085, data_02_0086, data_02_0087, data_02_0088, data_02_0089, data_02_0090, data_02_0091, data_02_0092, data_02_0093.
Residual error before fit:
6.737384336276989e-07


In [14]:
#First refinement:
gonioref0.refine2()

Cost function before refinement: 6.737384336276989e-07
[ 7.20000000e-01  3.20000000e-02  4.00000000e-03  0.00000000e+00
 -8.27999445e+01  1.00000000e+00  1.70270825e+01]
     fun: 2.490988798391531e-11
     jac: array([ 1.58414573e-07,  2.91897065e-08,  5.74391333e-11, -6.39962716e-10,
        3.42479452e-11,  1.40698490e-07, -1.59997085e-11])
 message: 'Optimization terminated successfully.'
    nfev: 19
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([ 7.19994724e-01,  3.14085784e-02,  4.00000000e-03,  0.00000000e+00,
       -8.27999519e+01,  1.00000000e+00,  1.70270825e+01])
Cost function after refinement: 2.490988798391531e-11
GonioParam(dist=0.7199947243683663, poni1=0.03140857835160603, poni2=0.004, rot1=0.0, offset=-82.7999518865857, scale=1.0, nrj=17.027082549190933)
maxdelta on: poni1 (1) 0.032 --> 0.03140857835160603


array([ 7.19994724e-01,  3.14085784e-02,  4.00000000e-03,  0.00000000e+00,
       -8.27999519e+01,  1.00000000e+00,  1.70270825e+01])

In [15]:
#Here we extract all spectra for peaks,
# If there are as many peaks as expected from the theoritical LaB6. perform the assignment.

#Peaks from LaB6:
tths = LaB6.get_2th()

for i in range(peak_zero_end, ds.shape[0]):
    peak = peak_picking(name, i)
    ai=gonioref0.get_ai(get_position(i))
    tth = ai.array_from_unit(unit="2th_rad", scale=False)
    tth_low = tth[20]
    tth_hi = tth[-20]
    ttmin, ttmax = min(tth_low, tth_hi), max(tth_low, tth_hi)
    valid_peaks = numpy.logical_and(ttmin<=tths, tths<ttmax)
    cnt = valid_peaks.sum()
    if (len(peak) ==  cnt):    
        cp = ControlPoints(calibrant=LaB6, wavelength=wl)
        #revert the order of assignment if needed !!
        if tth_hi < tth_low:
            peak = peak[-1::-1]
        for p, r in zip(peak, numpy.where(valid_peaks)[0]):
            #print(p,r)
            cp.append([p], ring=r)
        img = ds[i].reshape((-1,1))
        sg = gonioref0.new_geometry("%s_%04i"%(name,i), 
                                    image=img, 
                                    metadata=i, 
                                    control_points=cp, 
                                    calibrant=LaB6)
        sg.geometry_refinement.data = numpy.array(cp.getList())
        #print(sg.label, len(sg.geometry_refinement.data))

#print(gonioref0)
print(" Number of peaks found and used for refinement")
print(sum([len(sg.geometry_refinement.data) for sg in gonioref0.single_geometries.values()]))
print("Residual error before fitting: ", gonioref0.chi2())

 Number of peaks found and used for refinement
1203
Residual error before fitting:  3.118672809442448e-06


In [16]:
gonioref0.refine2()

Cost function before refinement: 3.118672809442448e-06
[ 7.19994724e-01  3.14085784e-02  4.00000000e-03  0.00000000e+00
 -8.27999519e+01  1.00000000e+00  1.70270825e+01]
     fun: 2.734250432678472e-06
     jac: array([-5.58224542e-08, -3.65304231e-10,  3.63058044e-06, -2.62495314e-06,
        6.31729336e-10,  1.53969062e-04,  9.87797637e-06])
 message: 'Optimization terminated successfully.'
    nfev: 65
     nit: 7
    njev: 7
  status: 0
 success: True
       x: array([ 7.23095347e-01,  3.18453768e-02,  4.00000000e-03,  0.00000000e+00,
       -8.27999466e+01,  1.00000000e+00,  1.70270825e+01])
Cost function after refinement: 2.734250432678472e-06
GonioParam(dist=0.7230953467494027, poni1=0.03184537681912273, poni2=0.004, rot1=0.0, offset=-82.79994661694376, scale=1.0, nrj=17.027082549190933)
maxdelta on: dist (0) 0.7199947243683663 --> 0.7230953467494027


array([ 7.23095347e-01,  3.18453768e-02,  4.00000000e-03,  0.00000000e+00,
       -8.27999466e+01,  1.00000000e+00,  1.70270825e+01])

In [17]:
gonioref0.set_bounds("poni1", -1, 1)
gonioref0.set_bounds("poni2", -1, 1)
gonioref0.set_bounds("rot1", -1, 1)
gonioref0.set_bounds("scale", 0.9, 1.1)
gonioref0.refine2()

Cost function before refinement: 2.734250432678472e-06
[ 7.23095347e-01  3.18453768e-02  4.00000000e-03  0.00000000e+00
 -8.27999466e+01  1.00000000e+00  1.70270825e+01]
     fun: 2.6891723022540698e-06
     jac: array([-5.14464347e-07, -1.24120874e-07,  5.95670514e-07, -4.28660144e-07,
       -9.28110921e-10, -8.53860058e-08, -1.60467593e-06])
 message: 'Optimization terminated successfully.'
    nfev: 36
     nit: 4
    njev: 4
  status: 0
 success: True
       x: array([ 7.23096248e-01,  3.20537111e-02,  3.98485468e-03,  1.09479202e-05,
       -8.27999440e+01,  9.99415047e-01,  1.70270825e+01])
Cost function after refinement: 2.6891723022540698e-06
GonioParam(dist=0.7230962484347616, poni1=0.03205371109950758, poni2=0.003984854679408886, rot1=1.0947920188414968e-05, offset=-82.7999439896301, scale=0.9994150470263444, nrj=17.027082549190933)
maxdelta on: scale (5) 1.0 --> 0.9994150470263444


array([ 7.23096248e-01,  3.20537111e-02,  3.98485468e-03,  1.09479202e-05,
       -8.27999440e+01,  9.99415047e-01,  1.70270825e+01])

In [18]:
# Perform the azimuthal intgration of all data for the first module:

mg = gonioref0.get_mg(position)
mg.radial_range = (0, 95)
images = [i.reshape(-1, 1) for i in ds]
res_mg = mg.integrate1d(images, 50000)
results={name: res_mg}
print(results)

{'data_02': (array([9.50000113e-04, 2.85000011e-03, 4.75000011e-03, ...,
       9.49952500e+01, 9.49971500e+01, 9.49990500e+01]), array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
       1.27163417e+08, 1.30591594e+08, 1.33665235e+08]))}


In [19]:
# Plot the integrated pattern vs expected peak positions:

LaB6_new = get_calibrant("LaB6")
LaB6_new.wavelength = hc/gonioref0.param[-1]*1e-10
p = jupyter.plot1d(res_mg, calibrant=LaB6_new)
p.figure.show()

/mntdirect/_scisoft/users/jupyter/jupy35/lib/python3.5/site-packages/matplotlib/__init__.py:932: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


FigureCanvasNbAgg()

In [20]:
#Peak profile function based on a bilinear interpolations: 

def calc_fwhm(integrate_result, calibrant, tth_min=None, tth_max=None):
    "calculate the tth position and FWHM for each peak"
    delta = integrate_result.intensity[1:] - integrate_result.intensity[:-1]
    maxima = numpy.where(numpy.logical_and(delta[:-1]>0, delta[1:]<0))[0]
    minima = numpy.where(numpy.logical_and(delta[:-1]<0, delta[1:]>0))[0]
    maxima += 1
    minima += 1
    tth = []
    FWHM = []
    if tth_min is None:
        tth_min = integrate_result.radial[0]
    if tth_max is None:
        tth_max = integrate_result.radial[-1]
    for tth_rad in calibrant.get_2th():
        tth_deg = tth_rad*integrate_result.unit.scale
        if (tth_deg<=tth_min) or (tth_deg>=tth_max):
            continue
        idx_theo = abs(integrate_result.radial-tth_deg).argmin()
        id0_max = abs(maxima-idx_theo).argmin()
        id0_min = abs(minima-idx_theo).argmin()
        I_max = integrate_result.intensity[maxima[id0_max]]
        I_min = integrate_result.intensity[minima[id0_min]]
        tth_maxi = integrate_result.radial[maxima[id0_max]]
        I_thres = (I_max + I_min)/2.0
        if minima[id0_min]>maxima[id0_max]:
            if id0_min == 0:
                min_lo = integrate_result.radial[0]
            else:
                min_lo = integrate_result.radial[minima[id0_min-1]]
            min_hi = integrate_result.radial[minima[id0_min]]
        else:
            if id0_min == len(minima) -1:
                min_hi = integrate_result.radial[-1]
            else:
                min_hi = integrate_result.radial[minima[id0_min+1]]
            min_lo = integrate_result.radial[minima[id0_min]]
            
        f = interp1d(integrate_result.radial, integrate_result.intensity-I_thres)
        try:
            tth_lo = bisect(f, min_lo, tth_maxi)
            tth_hi = bisect(f, tth_maxi, min_hi)
        except:
            pass
        else:
            FWHM.append(tth_hi-tth_lo)
            tth.append(tth_deg)
    return tth, FWHM
    


In [21]:
# Peak error:

def calc_peak_error(integrate_result, calibrant, tth_min=10, tth_max=95):
    "calculate the tth position and FWHM for each peak"
    peaks = find_peaks_cwt(integrate_result.intensity, [10])
    df = numpy.gradient(integrate_result.intensity)
    d2f = numpy.gradient(df)
    bad = d2f==0
    d2f[bad] = 1e-10
    cor = df / d2f
    print((abs(cor)>1).sum())
    cor[abs(cor)>1] = 0
    cor[bad] = 0
    got = numpy.interp(peaks-cor[peaks], 
                       numpy.arange(len(integrate_result.radial)), 
                       integrate_result.radial)
    mask = numpy.logical_and(got>=tth_min,
                             got<=tth_max)
    got = got[mask]
    target = numpy.array(calibrant.get_2th())*integrate_result.unit.scale
    mask = numpy.logical_and(target>=tth_min,
                             target<=tth_max)
    target = target[mask]
    print(len(got), len(target))
    d2 = distance_matrix(target.reshape(-1, 1 ),
                         got.reshape(-1, 1), p=1)
    
    return target, target-got[d2.argmin(axis=-1)]


In [22]:
fig, ax = plt.subplots()
ax.plot(*calc_fwhm(res_mg, LaB6_new), "o", label="FWHM")
ax.plot(*calc_peak_error(res_mg, LaB6_new), "o", label="offset")
ax.set_title("Peak shape & error as function of the angle")
ax.set_xlabel(res_mg.unit.label)
ax.legend()
fig.show()

/mntdirect/_scisoft/users/jupyter/jupy35/lib/python3.5/site-packages/matplotlib/__init__.py:932: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


29209
81 60


FigureCanvasNbAgg()

## Module 1 

We can apply the same procdure for the second module ... and try to rationalize the procedure.

In [23]:
module_id = 1
name = ds_names[module_id]
ds = data[name]
zero_pos = 64
frame_start = 103
frame_stop = 123

In [24]:
param1 = {"dist": 0.72, 
          "poni1": 640*50e-6, 
          "poni2": 4e-3, 
          "rot1":0, 
          "offset": -get_position(zero_pos), 
          "scale":1, 
          "nrj": nrj}

#Lock enegy for now and a couple of other parameters
bounds1 = {"nrj": (nrj, nrj),
           "dist": (0.7, 0.8),
           "poni2": (4e-3, 4e-3),
           "rot1": (0,0),
           "scale":(1,1), }

gonioref1 = GoniometerRefinement(param1, 
                                 get_position, 
                                 trans, 
                                 detector=modules[name], 
                                 wavelength=wl, 
                                 bounds=bounds1
                                 )
print(gonioref1)
goniometers[name]=gonioref1

GoniometerRefinement with 0 geometries labeled: .


In [25]:
#Exctract frames with peak#0
for i in range(frame_start, frame_stop):
    cp = ControlPoints(calibrant=LaB6, wavelength=wl)
    peak = peak_picking(name, i)
    if len(peak)!=1: 
        continue
    cp.append([peak[0]], ring=0)
    img = (ds[i]).reshape((-1,1))
    sg = gonioref1.new_geometry("%s_%04i"%(name,i), 
                                image=img, 
                                metadata=i, 
                                control_points=cp, 
                                calibrant=LaB6)
    sg.geometry_refinement.data = numpy.array(cp.getList())

print(gonioref1)
print(gonioref1.chi2())

GoniometerRefinement with 20 geometries labeled: data_03_0103, data_03_0104, data_03_0105, data_03_0106, data_03_0107, data_03_0108, data_03_0109, data_03_0110, data_03_0111, data_03_0112, data_03_0113, data_03_0114, data_03_0115, data_03_0116, data_03_0117, data_03_0118, data_03_0119, data_03_0120, data_03_0121, data_03_0122.
1.4524700202830549e-06


In [26]:
gonioref1.refine2()

Cost function before refinement: 1.4524700202830549e-06
[ 7.20000000e-01  3.20000000e-02  4.00000000e-03  0.00000000e+00
 -7.72000000e+01  1.00000000e+00  1.70270825e+01]
     fun: 2.3431861472723482e-11
     jac: array([ 1.37377624e-07,  1.03462536e-07,  2.38592839e-09, -2.23238045e-09,
        9.89688559e-10,  1.85837277e-07, -5.81633804e-10])
 message: 'Optimization terminated successfully.'
    nfev: 19
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([ 7.20006378e-01,  3.28683965e-02,  4.00000000e-03,  0.00000000e+00,
       -7.71999891e+01,  1.00000000e+00,  1.70270825e+01])
Cost function after refinement: 2.3431861472723482e-11
GonioParam(dist=0.7200063780313144, poni1=0.03286839651397956, poni2=0.004, rot1=0.0, offset=-77.19998908850266, scale=1.0, nrj=17.027082549190933)
maxdelta on: poni1 (1) 0.032 --> 0.03286839651397956


array([ 7.20006378e-01,  3.28683965e-02,  4.00000000e-03,  0.00000000e+00,
       -7.71999891e+01,  1.00000000e+00,  1.70270825e+01])

In [27]:
#Exctract all frames with peak>0
tths = LaB6.get_2th()
#print(tths)
for i in range(frame_stop, ds.shape[0]):
    frame_name = "%s_%04i"%(name, i)
    if frame_name in gonioref1.single_geometries:
        continue
    peak = peak_picking(name, i)
    ai=gonioref1.get_ai(get_position(i))
    tth = ai.array_from_unit(unit="2th_rad", scale=False)
    tth_low = tth[20]
    tth_hi = tth[-20]
    ttmin, ttmax = min(tth_low, tth_hi), max(tth_low, tth_hi)
    valid_peaks = numpy.logical_and(ttmin<=tths, tths<ttmax)
    cnt = valid_peaks.sum()
    if (len(peak) ==  cnt) and cnt>0:    
        cp = ControlPoints(calibrant=LaB6, wavelength=wl)
        #revert the order of assignment if needed !!
        if tth_hi < tth_low:
            peak = peak[-1::-1]
        for p, r in zip(peak, numpy.where(valid_peaks)[0]):
            cp.append([p], ring=r)
        img = ds[i].reshape((-1,1))
        sg = gonioref1.new_geometry(frame_name, 
                                    image=img, 
                                    metadata=i, 
                                    control_points=cp, 
                                    calibrant=LaB6)
        sg.geometry_refinement.data = numpy.array(cp.getList())
        #print(frame_name, len(sg.geometry_refinement.data))

print(" Number of peaks found and used for refinement")
print(sum([len(sg.geometry_refinement.data) for sg in gonioref1.single_geometries.values()]))
print("Residual error before fitting: ", gonioref1.chi2())

 Number of peaks found and used for refinement
1183
Residual error before fitting:  6.334769973618362e-07


In [28]:
gonioref1.refine2()
gonioref1.set_bounds("poni1", -1, 1)
gonioref1.set_bounds("poni2", -1, 1)
gonioref1.set_bounds("rot1", -1, 1)
gonioref1.set_bounds("scale", 0.9, 1.1)
gonioref1.refine2()

Cost function before refinement: 6.334769973618362e-07
[ 7.20006378e-01  3.28683965e-02  4.00000000e-03  0.00000000e+00
 -7.71999891e+01  1.00000000e+00  1.70270825e+01]
     fun: 1.3797542164386038e-07
     jac: array([-4.62580205e-07,  2.40834463e-08,  5.10766142e-06, -3.67570741e-06,
        1.35589318e-11,  2.65546301e-04,  1.99388955e-05])
 message: 'Optimization terminated successfully.'
    nfev: 19
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([ 7.20006340e-01,  3.33754761e-02,  4.00000000e-03,  0.00000000e+00,
       -7.71999827e+01,  1.00000000e+00,  1.70270825e+01])
Cost function after refinement: 1.3797542164386038e-07
GonioParam(dist=0.7200063395001164, poni1=0.033375476115602126, poni2=0.004, rot1=0.0, offset=-77.19998271232215, scale=1.0, nrj=17.027082549190933)
maxdelta on: poni1 (1) 0.03286839651397956 --> 0.033375476115602126
Cost function before refinement: 1.3797542164386038e-07
[ 7.20006340e-01  3.33754761e-02  4.00000000e-03  0.00000000e+00
 -

array([ 7.20682081e-01,  3.36782425e-02,  4.05801126e-03, -4.44860018e-05,
       -7.71999788e+01,  9.98967223e-01,  1.70270825e+01])

In [29]:
mg1 = gonioref1.get_mg(position)
mg1.radial_range = (0, 95)
images = [i.reshape(-1, 1) for i in data[name]]
res_mg1 = mg1.integrate1d(images, 50000)
results[name] = res_mg1

area_pixel=0.04011793214486481 area_sum=0.05472285778563535, Error= -0.3640498116411517
area_pixel=0.06755918226705582 area_sum=0.07017344439747103, Error= -0.038695881783785545
area_pixel=0.06470384723321665 area_sum=0.06854713835898787, Error= -0.05939818558112286
area_pixel=0.038258287348172715 area_sum=0.05426051394830549, Error= -0.4182682422373795
area_pixel=0.06290299608614092 area_sum=0.06750646087344028, Error= -0.07318355362589196
area_pixel=0.06866181632504009 area_sum=0.0710979827571788, Error= -0.03548065813764767
area_pixel=0.045481003274128184 area_sum=0.05690843539150094, Error= -0.2512572567604998
area_pixel=0.051372066242729275 area_sum=0.060437735665688296, Error= -0.17647079601829507
area_pixel=0.08412331180916155 area_sum=0.08447601010651973, Error= -0.004192634476377893
area_pixel=0.07467387290401106 area_sum=0.07603231757541912, Error= -0.018191699701370204
area_pixel=0.0624099895059711 area_sum=0.0673026438866445, Error= -0.07839537259023732
area_pixel=0.0419590

In [30]:
LaB6_new = get_calibrant("LaB6")
LaB6_new.wavelength = hc/gonioref1.param[-1]*1e-10
p = jupyter.plot1d(res_mg1, calibrant=LaB6_new)
p.figure.show()

/mntdirect/_scisoft/users/jupyter/jupy35/lib/python3.5/site-packages/matplotlib/__init__.py:932: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


FigureCanvasNbAgg()

In [31]:
fig, ax = plt.subplots()
ax.plot(*calc_fwhm(res_mg1, LaB6_new), "o", label="FWHM")
ax.plot(*calc_peak_error(res_mg1, LaB6_new, 10, 88), "o", label="error")
ax.set_title("Peak shape & error as function of the angle")
ax.set_xlabel(res_mg.unit.label)
ax.legend()
fig.show()

/mntdirect/_scisoft/users/jupyter/jupy35/lib/python3.5/site-packages/matplotlib/__init__.py:932: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


27712
72 53


FigureCanvasNbAgg()

## All other Modules

We define now an automatic procedure for any module. 
The detection used 3 parameter visually extracted from the Figure1: 

* zero_pos: the frame where the beam-stop is in the center of the module
* frame_start: the frame where the first peak of LaB6 appears (positive)
* frame_stop: the frame where the second peak of LaB6 appears (positive)

This is enough for boot-strapping the goniometer configuration.

In [32]:
def add_module(name,
               zero_pos,
               frame_start,
               frame_stop,
                ):
    ds = data[name]
    param = {"dist": 0.72, 
          "poni1": 640*50e-6, 
          "poni2": 4e-3, 
          "rot1":0, 
          "offset": -get_position(zero_pos), 
          "scale":1, 
          "nrj": nrj}

    #Lock enegy for now and a couple of other parameters
    bounds = {"nrj": (nrj, nrj),
              "dist": (0.7, 0.8),
              "poni2": (4e-3, 4e-3),
              "rot1": (0,0),
              "scale": (1,1)}

    gonioref = GoniometerRefinement(param, 
                                    get_position, 
                                    trans, 
                                    detector=modules[name], 
                                    wavelength=wl, 
                                    bounds=bounds
                                      )
    goniometers[name] = gonioref
    
    for i in range(frame_start, frame_stop):
        cp = ControlPoints(calibrant=LaB6, wavelength=wl)
        peak = peak_picking(name, i)
        if len(peak)!=1: 
            continue
        cp.append([peak[0]], ring=0)
        img = (ds[i]).reshape((-1,1))
        sg = gonioref.new_geometry("%s_%04i"%(name, i), 
                                    image=img, 
                                    metadata=i, 
                                    control_points=cp, 
                                    calibrant=LaB6)
        sg.geometry_refinement.data = numpy.array(cp.getList())

    print(gonioref.chi2())
    gonioref.refine2()
        
    tths = LaB6.get_2th()
    #print(tths)
    for i in range(frame_stop, ds.shape[0]):
        frame_name = "%s_%04i"%(name, i)
        if frame_name in gonioref.single_geometries:
            continue
        peak = peak_picking(name, i)
        ai=gonioref.get_ai(get_position(i))
        tth = ai.array_from_unit(unit="2th_rad", scale=False)
        tth_low = tth[20]
        tth_hi = tth[-20]
        ttmin, ttmax = min(tth_low, tth_hi), max(tth_low, tth_hi)
        valid_peaks = numpy.logical_and(ttmin<=tths, tths<ttmax)
        cnt = valid_peaks.sum()
        if (len(peak) ==  cnt) and cnt>0:    
            cp = ControlPoints(calibrant=LaB6, wavelength=wl)
            #revert the order of assignment if needed !!
            if tth_hi < tth_low:
                peak = peak[-1::-1]

            for p, r in zip(peak, numpy.where(valid_peaks)[0]):
                cp.append([p], ring=r)
            img = (ds[i]).reshape((-1,1))
            sg = gonioref.new_geometry(frame_name, 
                                        image=img, 
                                        metadata=i, 
                                        control_points=cp, 
                                        calibrant=LaB6)
            sg.geometry_refinement.data = numpy.array(cp.getList())
            #print(frame_name, len(sg.geometry_refinement.data))


    print(" Number of peaks found and used for refinement")
    print(sum([len(sg.geometry_refinement.data) for sg in gonioref.single_geometries.values()]))

    gonioref.refine2()
    gonioref.set_bounds("poni1", -1, 1)
    gonioref.set_bounds("poni2", -1, 1)
    gonioref.set_bounds("rot1", -1, 1)
    gonioref.set_bounds("scale", 0.9, 1.1)
    gonioref.refine2()
    
    mg = gonioref.get_mg(position)
    mg.radial_range = (0, 95)
    images = [i.reshape(-1, 1) for i in ds]
    res_mg = mg.integrate1d(images, 50000)
    results[name] = res_mg
    
    LaB6_new = get_calibrant("LaB6")
    LaB6_new.wavelength = hc/gonioref.param[-1]*1e-10
    p = jupyter.plot1d(res_mg, calibrant=LaB6_new)
    p.figure.show()
    
    fig, ax = plt.subplots()
    ax.plot(*calc_fwhm(res_mg, LaB6_new), "o", label="FWHM")
    ax.plot(*calc_peak_error(res_mg, LaB6_new, 10, 89), "o", label="error")
    ax.set_title("Peak shape & error as function of the angle")
    ax.set_xlabel(res_mg.unit.label)
    ax.legend()
    fig.show()

In [33]:
add_module(ds_names[2],
           92,
           131,
           151)

8.406338771704969e-06
Cost function before refinement: 8.406338771704969e-06
[ 7.20000000e-01  3.20000000e-02  4.00000000e-03  0.00000000e+00
 -7.15999445e+01  1.00000000e+00  1.70270825e+01]
     fun: 1.808572002652971e-11
     jac: array([ 4.57424569e-08,  8.17041295e-07,  2.47626008e-08, -1.79775418e-08,
        9.91633086e-09,  6.60392633e-07, -5.86643086e-09])
 message: 'Optimization terminated successfully.'
    nfev: 19
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([ 7.20018961e-01,  3.40893083e-02,  4.00000000e-03,  0.00000000e+00,
       -7.15999182e+01,  1.00000000e+00,  1.70270825e+01])
Cost function after refinement: 1.808572002652971e-11
GonioParam(dist=0.7200189605020697, poni1=0.03408930832642036, poni2=0.004, rot1=0.0, offset=-71.59991818229837, scale=1.0, nrj=17.027082549190933)
maxdelta on: poni1 (1) 0.032 --> 0.03408930832642036
 Number of peaks found and used for refinement
1093
Cost function before refinement: 5.55731697942323e-07
[ 7.20018961e

/mntdirect/_scisoft/users/jupyter/jupy35/lib/python3.5/site-packages/matplotlib/__init__.py:932: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


FigureCanvasNbAgg()

25667
180 54


FigureCanvasNbAgg()

In [34]:
add_module(ds_names[3],
           121,
           159,
           179)

1.0825408441330163e-06
Cost function before refinement: 1.0825408441330163e-06
[ 7.20000000e-01  3.20000000e-02  4.00000000e-03  0.00000000e+00
 -6.57999444e+01  1.00000000e+00  1.70270825e+01]
     fun: 2.4921939380829473e-11
     jac: array([-8.60015175e-08,  7.99451827e-08,  1.64794931e-09, -8.07493243e-10,
        6.18853503e-10, -1.33401994e-08, -3.62117520e-10])
 message: 'Optimization terminated successfully.'
    nfev: 19
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([ 7.20006745e-01,  3.27496622e-02,  4.00000000e-03,  0.00000000e+00,
       -6.57999350e+01,  1.00000000e+00,  1.70270825e+01])
Cost function after refinement: 2.4921939380829473e-11
GonioParam(dist=0.7200067449851295, poni1=0.032749662244436796, poni2=0.004, rot1=0.0, offset=-65.79993502488618, scale=1.0, nrj=17.027082549190933)
maxdelta on: poni1 (1) 0.032 --> 0.032749662244436796
 Number of peaks found and used for refinement
978
Cost function before refinement: 4.705197877733383e-07
[ 7.200

/mntdirect/_scisoft/users/jupyter/jupy35/lib/python3.5/site-packages/matplotlib/__init__.py:932: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


FigureCanvasNbAgg()

24026
384 54


FigureCanvasNbAgg()

In [35]:
add_module(ds_names[4],
           150,
           188,
           208)

2.3192859672732885e-07
Cost function before refinement: 2.3192859672732885e-07
[ 7.20000000e-01  3.20000000e-02  4.00000000e-03  0.00000000e+00
 -6.00000556e+01  1.00000000e+00  1.70270825e+01]
     fun: 2.0856624751999954e-10
     jac: array([-2.72031959e-07,  2.34632462e-08, -3.13650477e-11,  1.14575097e-09,
       -9.14368789e-11, -1.88744526e-07,  5.83778581e-11])
 message: 'Optimization terminated successfully.'
    nfev: 19
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([ 7.19996884e-01,  3.16531473e-02,  4.00000000e-03,  0.00000000e+00,
       -6.00000599e+01,  1.00000000e+00,  1.70270825e+01])
Cost function after refinement: 2.0856624751999954e-10
GonioParam(dist=0.719996884236354, poni1=0.031653147292955666, poni2=0.004, rot1=0.0, offset=-60.00005992106343, scale=1.0, nrj=17.027082549190933)
maxdelta on: poni1 (1) 0.032 --> 0.031653147292955666
 Number of peaks found and used for refinement
864
Cost function before refinement: 3.883355149577502e-07
[ 7.1999

/mntdirect/_scisoft/users/jupyter/jupy35/lib/python3.5/site-packages/matplotlib/__init__.py:932: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


FigureCanvasNbAgg()

22258
578 54


FigureCanvasNbAgg()

In [36]:
add_module(ds_names[5],
           178,
           216,
           236)

1.496585538687667e-06
Cost function before refinement: 1.496585538687667e-06
[ 7.20000000e-01  3.20000000e-02  4.00000000e-03  0.00000000e+00
 -5.43998333e+01  1.00000000e+00  1.70270825e+01]
     fun: 2.0171535150920134e-10
     jac: array([-2.79014458e-07,  1.12723053e-07,  2.76456642e-09, -8.39426390e-10,
        1.01678678e-09, -1.46581687e-07, -5.97716569e-10])
 message: 'Optimization terminated successfully.'
    nfev: 19
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([ 7.20008138e-01,  3.28813990e-02,  4.00000000e-03,  0.00000000e+00,
       -5.43998223e+01,  1.00000000e+00,  1.70270825e+01])
Cost function after refinement: 2.0171535150920134e-10
GonioParam(dist=0.7200081378039512, poni1=0.03288139900481467, poni2=0.004, rot1=0.0, offset=-54.399822256631786, scale=1.0, nrj=17.027082549190933)
maxdelta on: poni1 (1) 0.032 --> 0.03288139900481467
 Number of peaks found and used for refinement
754
Cost function before refinement: 3.084426464441637e-07
[ 7.200081

/mntdirect/_scisoft/users/jupyter/jupy35/lib/python3.5/site-packages/matplotlib/__init__.py:932: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


FigureCanvasNbAgg()

20560
758 54


FigureCanvasNbAgg()

In [37]:
add_module(ds_names[6],
           207,
           245,
           266)

7.414763745399852e-08
Cost function before refinement: 7.414763745399852e-08
[ 7.20000000e-01  3.20000000e-02  4.00000000e-03  0.00000000e+00
 -4.85998889e+01  1.00000000e+00  1.70270825e+01]
     fun: 5.6513401976066356e-11
     jac: array([-2.81311437e-07,  1.90457697e-08, -2.47022619e-10,  1.33814411e-09,
       -1.23389936e-10, -2.00557892e-07,  7.73080818e-11])
 message: 'Optimization terminated successfully.'
    nfev: 19
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([ 7.19998572e-01,  3.18038587e-02,  4.00000000e-03,  0.00000000e+00,
       -4.85998914e+01,  1.00000000e+00,  1.70270825e+01])
Cost function after refinement: 5.6513401976066356e-11
GonioParam(dist=0.719998571752675, poni1=0.031803858748058605, poni2=0.004, rot1=0.0, offset=-48.599891358709684, scale=1.0, nrj=17.027082549190933)
maxdelta on: poni1 (1) 0.032 --> 0.031803858748058605
 Number of peaks found and used for refinement
626
Cost function before refinement: 2.383350348017308e-07
[ 7.19998

/mntdirect/_scisoft/users/jupyter/jupy35/lib/python3.5/site-packages/matplotlib/__init__.py:932: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


FigureCanvasNbAgg()

18647
952 54


FigureCanvasNbAgg()

In [38]:
add_module(ds_names[7],
           236,
           273,
           293)

4.37748174147368e-06
Cost function before refinement: 4.37748174147368e-06
[ 7.20000000e-01  3.20000000e-02  4.00000000e-03  0.00000000e+00
 -4.27998889e+01  1.00000000e+00  1.70270825e+01]
     fun: 9.109058243486396e-11
     jac: array([-3.53771074e-07,  3.50167703e-09, -7.17406458e-10,  1.96667227e-09,
       -3.65232464e-10, -2.61708398e-07,  2.20481224e-10])
 message: 'Optimization terminated successfully.'
    nfev: 19
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([ 7.19986034e-01,  3.04925065e-02,  4.00000000e-03,  0.00000000e+00,
       -4.27999078e+01,  1.00000000e+00,  1.70270825e+01])
Cost function after refinement: 9.109058243486396e-11
GonioParam(dist=0.7199860338026991, poni1=0.030492506521307684, poni2=0.004, rot1=0.0, offset=-42.79990784447931, scale=1.0, nrj=17.027082549190933)
maxdelta on: poni1 (1) 0.032 --> 0.030492506521307684
 Number of peaks found and used for refinement
543
Cost function before refinement: 1.9115723976987742e-07
[ 7.19986034

/mntdirect/_scisoft/users/jupyter/jupy35/lib/python3.5/site-packages/matplotlib/__init__.py:932: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


FigureCanvasNbAgg()

16809
1107 54


FigureCanvasNbAgg()

In [39]:
add_module(ds_names[8],
           264,
           302,
           322)

9.734044802496035e-08
Cost function before refinement: 9.734044802496035e-08
[ 7.20000000e-01  3.20000000e-02  4.00000000e-03  0.00000000e+00
 -3.72000000e+01  1.00000000e+00  1.70270825e+01]
     fun: 1.3838347147363678e-10
     jac: array([-5.98673543e-07,  2.23605236e-08, -1.03159942e-10,  2.50403027e-09,
       -2.11195058e-10, -4.33428498e-07,  1.29258940e-10])
 message: 'Optimization terminated successfully.'
    nfev: 19
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([ 7.19998053e-01,  3.17753501e-02,  4.00000000e-03,  0.00000000e+00,
       -3.72000028e+01,  1.00000000e+00,  1.70270825e+01])
Cost function after refinement: 1.3838347147363678e-10
GonioParam(dist=0.7199980533232613, poni1=0.03177535006475983, poni2=0.004, rot1=0.0, offset=-37.20000282727144, scale=1.0, nrj=17.027082549190933)
maxdelta on: poni1 (1) 0.032 --> 0.03177535006475983
 Number of peaks found and used for refinement
454
Cost function before refinement: 1.430842008138168e-07
[ 7.1999805

/mntdirect/_scisoft/users/jupyter/jupy35/lib/python3.5/site-packages/matplotlib/__init__.py:932: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


FigureCanvasNbAgg()

16876
1111 54


FigureCanvasNbAgg()

In [40]:
len(goniometers)

9

In [41]:
# print all the parameters to be able to compare them visually
goniometers["data_12"] = goniometers["data_11"]
for name in ds_names:
    print(name, *["%8.4e"%i for i in goniometers[name].param])

data_02 7.2310e-01 3.2054e-02 3.9849e-03 1.0948e-05 -8.2800e+01 9.9942e-01 1.7027e+01
data_03 7.2068e-01 3.3678e-02 4.0580e-03 -4.4486e-05 -7.7200e+01 9.9897e-01 1.7027e+01
data_04 7.2069e-01 3.4825e-02 4.0432e-03 -3.3769e-05 -7.1600e+01 9.9898e-01 1.7027e+01
data_05 7.2048e-01 3.3410e-02 3.9779e-03 1.5910e-05 -6.5800e+01 9.9900e-01 1.7027e+01
data_07 7.2065e-01 3.2237e-02 3.9757e-03 1.7542e-05 -6.0000e+01 9.9901e-01 1.7027e+01
data_08 7.2053e-01 3.3395e-02 3.9730e-03 1.9484e-05 -5.4400e+01 9.9903e-01 1.7027e+01
data_09 7.2061e-01 3.2247e-02 3.9689e-03 2.2376e-05 -4.8600e+01 9.9904e-01 1.7027e+01
data_10 7.2071e-01 3.0863e-02 3.9652e-03 2.5070e-05 -4.2800e+01 9.9905e-01 1.7027e+01
data_11 7.2081e-01 3.2082e-02 3.9596e-03 2.9091e-05 -3.7200e+01 9.9906e-01 1.7027e+01
data_12 7.2081e-01 3.2082e-02 3.9596e-03 2.9091e-05 -3.7200e+01 9.9906e-01 1.7027e+01


## Use the negative part of the spectum ...

Until now, we used only the data where 2th >0 
For the last modules, this thows away half of the data.

We setup here a way to assign the peaks for the negative part of the spectrum.

In [42]:
name = ds_names[8]
print(name)
gonioref = goniometers[name]
print(gonioref)
ds = data[name]

data_11
GoniometerRefinement with 198 geometries labeled: data_11_0302, data_11_0303, data_11_0304, data_11_0305, data_11_0306, data_11_0307, data_11_0308, data_11_0309, data_11_0310, data_11_0311, data_11_0312, data_11_0313, data_11_0314, data_11_0315, data_11_0316, data_11_0317, data_11_0318, data_11_0319, data_11_0320, data_11_0321, data_11_0322, data_11_0323, data_11_0324, data_11_0325, data_11_0326, data_11_0327, data_11_0328, data_11_0329, data_11_0330, data_11_0331, data_11_0332, data_11_0333, data_11_0334, data_11_0335, data_11_0336, data_11_0337, data_11_0338, data_11_0339, data_11_0340, data_11_0341, data_11_0342, data_11_0343, data_11_0344, data_11_0345, data_11_0346, data_11_0347, data_11_0348, data_11_0349, data_11_0350, data_11_0351, data_11_0352, data_11_0353, data_11_0354, data_11_0355, data_11_0356, data_11_0357, data_11_0358, data_11_0359, data_11_0360, data_11_0361, data_11_0362, data_11_0363, data_11_0364, data_11_0365, data_11_0366, data_11_0367, data_11_0368, data

In [43]:
cutof = 198
if len(gonioref.single_geometries)>cutof:
    labels = list(gonioref.single_geometries.keys())
    for lbl in labels[cutof:]:
        print(lbl, gonioref.single_geometries.pop(lbl))

In [44]:
print(" Number of peaks found and used for refinement")
print(sum([len(sg.geometry_refinement.data) for sg in gonioref.single_geometries.values()]))

tths = LaB6.get_2th()
for i in range(ds.shape[0]):
    frame_name = "%s_%04i"%(name, i)
    if frame_name in gonioref.single_geometries:
            continue
    peak = peak_picking(name, i)
    ai=gonioref.get_ai(get_position(i))
    tth = ai.array_from_unit(unit="2th_rad", scale=False)
    tth_low = tth[20]
    tth_hi = tth[-20]
    ttmin, ttmax = min(tth_low, tth_hi), max(tth_low, tth_hi)
    valid_peaks = numpy.logical_and(ttmin<=tths, tths<ttmax)
    cnt = valid_peaks.sum()
    if (len(peak) ==  cnt) and cnt>0:    
        cp = ControlPoints(calibrant=LaB6, wavelength=wl)
        #revert the order of assignment if needed !!
        if tth_hi < tth_low:
            peak = peak[-1::-1]
        for p, r in zip(peak, numpy.where(valid_peaks)[0]):
            cp.append([p], ring=r)
        img = ds[i].reshape((-1,1))
        sg = gonioref.new_geometry(frame_name, 
                                    image=img, 
                                    metadata=i, 
                                    control_points=cp, 
                                    calibrant=LaB6)
        sg.geometry_refinement.data = numpy.array(cp.getList())
        #print(frame_name, len(sg.geometry_refinement.data))


print(" Number of peaks found and used for refinement")
print(sum([len(sg.geometry_refinement.data) for sg in gonioref.single_geometries.values()]))


 Number of peaks found and used for refinement
454
 Number of peaks found and used for refinement
1006


In [45]:
gonioref.param = numpy.array([ 7.20810790e-01,  3.20817605e-02,  3.95963325e-03,  2.90908690e-05,
 -3.71999988e+01,  9.99058845e-01,  1.70270825e+01])
gonioref.set_bounds("dist", 0.719, 0.722)
print(gonioref.chi2())
gonioref.refine2()

4.706922201004137e-09
Cost function before refinement: 4.706922201004137e-09
[ 7.20810790e-01  3.20817605e-02  3.95963325e-03  2.90908690e-05
 -3.71999988e+01  9.99058845e-01  1.70270825e+01]
     fun: 9.802741573719933e-10
     jac: array([-8.29340049e-07,  1.79787813e-08, -2.22576447e-07,  1.63791396e-07,
       -1.37364439e-10, -3.00381641e-08,  3.83178731e-08])
 message: 'Optimization terminated successfully.'
    nfev: 37
     nit: 4
    njev: 4
  status: 0
 success: True
       x: array([ 7.20813440e-01,  3.20849224e-02,  3.96153953e-03,  2.77060770e-05,
       -3.71999988e+01,  9.98991349e-01,  1.70270825e+01])
Cost function after refinement: 9.802741573719933e-10
GonioParam(dist=0.7208134396060545, poni1=0.03208492244228144, poni2=0.003961539529341621, rot1=2.770607695194935e-05, offset=-37.19999875911095, scale=0.9989913487177762, nrj=17.027082549190933)
maxdelta on: scale (5) 0.999058845 --> 0.9989913487177762


array([ 7.20813440e-01,  3.20849224e-02,  3.96153953e-03,  2.77060770e-05,
       -3.71999988e+01,  9.98991349e-01,  1.70270825e+01])

In [46]:
mg = gonioref.get_mg(position)
mg.radial_range = (0, 55)
images = [i.reshape(-1, 1) for i in data[name]]
res_mg = mg.integrate1d(images, 50000)
#results[name] = res_mg1

area_pixel=0.087116077029183 area_sum=0.08826425175192351, Error= -0.013179825835774046
area_pixel=0.09437982919091326 area_sum=0.09482188039672133, Error= -0.004683746618293709
area_pixel=0.09141781050882258 area_sum=0.09205052989663455, Error= -0.006921182910532562
area_pixel=0.099914213873042 area_sum=0.0999803125598409, Error= -0.0006615543898777575
area_pixel=0.08006542170439701 area_sum=0.08270949348660969, Error= -0.03302389128698582
area_pixel=0.0805270256009969 area_sum=0.08310831919799991, Error= -0.032054997409106584
area_pixel=0.09658809932440349 area_sum=0.09680466507299641, Error= -0.002242157678924418
area_pixel=0.07138542570604223 area_sum=0.07655014580204311, Error= -0.07234978351559695
area_pixel=0.08436896591885912 area_sum=0.08581985244955465, Error= -0.017196922054146083
area_pixel=0.09022642720770513 area_sum=0.09079300894250818, Error= -0.006279554143252871
area_pixel=0.07449146799122985 area_sum=0.07876427393764218, Error= -0.05735966898806962
area_pixel=0.07343

In [47]:
fig, ax = plt.subplots()
ax.plot(*calc_fwhm(res_mg, LaB6), "o", label="FWHM")
ax.plot(*calc_peak_error(res_mg, LaB6, 10, 55), "o", label="error")
ax.set_title("Peak shape & error as function of the angle")
ax.set_xlabel(res_mg.unit.label)
ax.legend()
fig.show()

/mntdirect/_scisoft/users/jupyter/jupy35/lib/python3.5/site-packages/matplotlib/__init__.py:932: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


31325
104 24


FigureCanvasNbAgg()

In [48]:
p = jupyter.plot1d(res_mg, calibrant=LaB6)
p.figure.show()

/mntdirect/_scisoft/users/jupyter/jupy35/lib/python3.5/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/mntdirect/_scisoft/users/jupyter/jupy35/lib/python3.5/site-packages/matplotlib/__init__.py:932: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


FigureCanvasNbAgg()

In [49]:
def complete_gonio(module_id=None, name=None):
    "Scan missing frames for un-indexed peaks"
    if name is None:
        name = ds_names[module_id]
    gonioref = goniometers[name]
    ds = data[name]
    print("Number of peaks previously found:",
           sum([len(sg.geometry_refinement.data) for sg in gonioref.single_geometries.values()]))

    tths = LaB6.get_2th()

    for i in range(ds.shape[0]):
        frame_name = "%s_%04i"%(name, i)
        if frame_name in gonioref.single_geometries:
                continue
        peak = peak_picking(name, i)
        ai=gonioref.get_ai(get_position(i))
        tth = ai.array_from_unit(unit="2th_rad", scale=False)
        tth_low = tth[20]
        tth_hi = tth[-20]
        ttmin, ttmax = min(tth_low, tth_hi), max(tth_low, tth_hi)
        valid_peaks = numpy.logical_and(ttmin<=tths, tths<ttmax)
        cnt = valid_peaks.sum()
        if (len(peak) ==  cnt) and cnt>0:    
            cp = ControlPoints(calibrant=LaB6, wavelength=wl)
            #revert the order of assignment if needed !!
            if tth_hi < tth_low:
                peak = peak[-1::-1]
            for p, r in zip(peak, numpy.where(valid_peaks)[0]):
                cp.append([p], ring=r)
            img = ds[i].reshape((-1,1))
            sg = gonioref.new_geometry(frame_name, 
                                        image=img, 
                                        metadata=i, 
                                        control_points=cp, 
                                        calibrant=LaB6)
            sg.geometry_refinement.data = numpy.array(cp.getList())
            #print(frame_name, len(sg.geometry_refinement.data))

    print("Number of peaks found after re-scan:",
            sum([len(sg.geometry_refinement.data) for sg in gonioref.single_geometries.values()]))
    return gonioref

In [50]:
gonio8 = complete_gonio(module_id=8)
gonio8.refine2()

Number of peaks previously found: 1006
Number of peaks found after re-scan: 1006
Cost function before refinement: 9.802741573719933e-10
[ 7.20813440e-01  3.20849224e-02  3.96153953e-03  2.77060770e-05
 -3.71999988e+01  9.98991349e-01  1.70270825e+01]
     fun: 9.802741573719933e-10
     jac: array([-8.29340049e-07,  1.79787813e-08, -2.22576447e-07,  1.63791396e-07,
       -1.37364439e-10, -3.00381641e-08,  3.83178731e-08])
 message: 'Optimization terminated successfully.'
    nfev: 9
     nit: 1
    njev: 1
  status: 0
 success: True
       x: array([ 7.20813440e-01,  3.20849224e-02,  3.96153953e-03,  2.77060770e-05,
       -3.71999988e+01,  9.98991349e-01,  1.70270825e+01])
Cost function after refinement: 9.802741573719933e-10
GonioParam(dist=0.7208134396060545, poni1=0.03208492244228144, poni2=0.003961539529341621, rot1=2.770607695194935e-05, offset=-37.19999875911095, scale=0.9989913487177762, nrj=17.027082549190933)
Restore wavelength and former parameters
GonioParam(dist=0.7208134

array([ 7.20813440e-01,  3.20849224e-02,  3.96153953e-03,  2.77060770e-05,
       -3.71999988e+01,  9.98991349e-01,  1.70270825e+01])

In [51]:
gonio7 = complete_gonio(module_id=7)

Number of peaks previously found: 543
Number of peaks found after re-scan: 1004


In [52]:
gonio7.refine2()

Cost function before refinement: 2.049750654664114e-09
[ 7.20707163e-01  3.08633971e-02  3.96520588e-03  2.50699030e-05
 -4.27999030e+01  9.99050597e-01  1.70270825e+01]
     fun: 6.982178014396509e-10
     jac: array([-6.61610503e-07,  1.82061135e-08, -2.31096076e-07,  1.69227578e-07,
       -1.05927128e-10, -2.98934004e-08,  2.67495021e-08])
 message: 'Optimization terminated successfully.'
    nfev: 37
     nit: 4
    njev: 4
  status: 0
 success: True
       x: array([ 7.20709219e-01,  3.08662496e-02,  3.96669042e-03,  2.39916666e-05,
       -4.27999029e+01,  9.99006679e-01,  1.70270825e+01])
Cost function after refinement: 6.982178014396509e-10
GonioParam(dist=0.7207092190734912, poni1=0.030866249621701404, poni2=0.0039666904208374675, rot1=2.3991666571782472e-05, offset=-42.799902936952186, scale=0.9990066787571606, nrj=17.027082549190933)
maxdelta on: scale (5) 0.99905059684665 --> 0.9990066787571606


array([ 7.20709219e-01,  3.08662496e-02,  3.96669042e-03,  2.39916666e-05,
       -4.27999029e+01,  9.99006679e-01,  1.70270825e+01])

In [53]:
gonio6 = complete_gonio(module_id=6)
gonio6.refine2()

Number of peaks previously found: 626
Number of peaks found after re-scan: 990
Cost function before refinement: 9.054048566564434e-10
[ 7.20609298e-01  3.22472627e-02  3.96893520e-03  2.23757106e-05
 -4.85998857e+01  9.99040899e-01  1.70270825e+01]
     fun: 6.093728642246349e-10
     jac: array([-8.02155920e-07,  3.63691515e-08, -2.33166253e-07,  1.71261021e-07,
        1.53140861e-10, -6.90587253e-08,  3.30203031e-08])
 message: 'Optimization terminated successfully.'
    nfev: 37
     nit: 4
    njev: 4
  status: 0
 success: True
       x: array([ 7.20611756e-01,  3.22490408e-02,  3.97006720e-03,  2.15500507e-05,
       -4.85998856e+01,  9.99018033e-01,  1.70270825e+01])
Cost function after refinement: 6.093728642246349e-10
GonioParam(dist=0.7206117564391414, poni1=0.03224904075932756, poni2=0.003970067202282311, rot1=2.1550050733043843e-05, offset=-48.599885636781195, scale=0.9990180333619139, nrj=17.027082549190933)
maxdelta on: scale (5) 0.9990408988314912 --> 0.9990180333619139


array([ 7.20611756e-01,  3.22490408e-02,  3.97006720e-03,  2.15500507e-05,
       -4.85998856e+01,  9.99018033e-01,  1.70270825e+01])

In [54]:
gonio5 = complete_gonio(module_id=5)
gonio5.refine2()

Number of peaks previously found: 754
Number of peaks found after re-scan: 1038
Cost function before refinement: 5.322312447273672e-10
[ 7.20528271e-01  3.33948197e-02  3.97295090e-03  1.94838596e-05
 -5.43998157e+01  9.99025942e-01  1.70270825e+01]
     fun: 5.247814297615004e-10
     jac: array([-6.54781369e-07,  1.29256630e-07, -1.62656541e-07,  1.19840950e-07,
        1.28911966e-09, -2.77499652e-07,  2.46511989e-08])
 message: 'Optimization terminated successfully.'
    nfev: 37
     nit: 4
    njev: 4
  status: 0
 success: True
       x: array([ 7.20530687e-01,  3.33953448e-02,  3.97363667e-03,  1.89799990e-05,
       -5.43998157e+01,  9.99021890e-01,  1.70270825e+01])
Cost function after refinement: 5.247814297615004e-10
GonioParam(dist=0.7205306865337416, poni1=0.033395344810677706, poni2=0.0039736366657854355, rot1=1.8979998986411344e-05, offset=-54.39981565735447, scale=0.999021890228638, nrj=17.027082549190933)
maxdelta on: scale (5) 0.999025941539747 --> 0.999021890228638


array([ 7.20530687e-01,  3.33953448e-02,  3.97363667e-03,  1.89799990e-05,
       -5.43998157e+01,  9.99021890e-01,  1.70270825e+01])

In [55]:
gonio4 = complete_gonio(module_id=4)
gonio4.refine2()

Number of peaks previously found: 864
Number of peaks found after re-scan: 1081
Cost function before refinement: 4.522981580650044e-10
[ 7.20645314e-01  3.22371238e-02  3.97565133e-03  1.75424155e-05
 -6.00000524e+01  9.99011856e-01  1.70270825e+01]
     fun: 4.329968320302878e-10
     jac: array([-3.23619143e-07, -1.33810126e-08, -8.07893403e-08,  5.95277891e-08,
       -4.92710775e-10,  3.16598865e-08,  5.83323419e-08])
 message: 'Optimization terminated successfully.'
    nfev: 37
     nit: 4
    njev: 4
  status: 0
 success: True
       x: array([ 7.20646423e-01,  3.22359951e-02,  3.97581932e-03,  1.74168753e-05,
       -6.00000525e+01,  9.99019702e-01,  1.70270825e+01])
Cost function after refinement: 4.329968320302878e-10
GonioParam(dist=0.7206464234341815, poni1=0.032235995133433754, poni2=0.0039758193176266145, rot1=1.7416875286579755e-05, offset=-60.00005245067518, scale=0.9990197021796172, nrj=17.027082549190933)
maxdelta on: scale (5) 0.999011855791219 --> 0.9990197021796172

array([ 7.20646423e-01,  3.22359951e-02,  3.97581932e-03,  1.74168753e-05,
       -6.00000525e+01,  9.99019702e-01,  1.70270825e+01])

In [56]:
gonio3 = complete_gonio(module_id=3)
gonio3.refine2()

Number of peaks previously found: 978
Number of peaks found after re-scan: 1156
Cost function before refinement: 7.053238687700532e-10
[ 7.20482931e-01  3.34099564e-02  3.97791142e-03  1.59099880e-05
 -6.57999266e+01  9.98999110e-01  1.70270825e+01]
     fun: 6.316819601296489e-10
     jac: array([-4.99814708e-07, -1.63155748e-08,  9.67096626e-09, -4.95315009e-09,
       -5.75364555e-10,  3.90058487e-08,  1.04654441e-07])
 message: 'Optimization terminated successfully.'
    nfev: 37
     nit: 4
    njev: 4
  status: 0
 success: True
       x: array([ 7.20484799e-01,  3.34074959e-02,  3.97770054e-03,  1.60543912e-05,
       -6.57999266e+01,  9.99015617e-01,  1.70270825e+01])
Cost function after refinement: 6.316819601296489e-10
GonioParam(dist=0.7204847991260711, poni1=0.03340749593129299, poni2=0.003977700542249465, rot1=1.605439120863723e-05, offset=-65.79992658922357, scale=0.9990156171464057, nrj=17.027082549190933)
maxdelta on: scale (5) 0.9989991098968561 --> 0.9990156171464057


array([ 7.20484799e-01,  3.34074959e-02,  3.97770054e-03,  1.60543912e-05,
       -6.57999266e+01,  9.99015617e-01,  1.70270825e+01])

In [57]:
gonio2 = complete_gonio(module_id=2)
gonio2.refine2()

Number of peaks previously found: 1093
Number of peaks found after re-scan: 1229
Cost function before refinement: 8.306533586355894e-10
[ 7.20686437e-01  3.48247495e-02  4.04316666e-03 -3.37686772e-05
 -7.15999088e+01  9.98982863e-01  1.70270825e+01]
     fun: 7.771891010372871e-10
     jac: array([ 9.15257278e-08, -1.05189468e-09,  5.61627546e-08, -4.08345480e-08,
       -4.25668376e-10,  2.85157377e-09,  1.17395008e-07])
 message: 'Optimization terminated successfully.'
    nfev: 37
     nit: 4
    njev: 4
  status: 0
 success: True
       x: array([ 7.20686041e-01,  3.48226304e-02,  4.04289725e-03, -3.35729680e-05,
       -7.15999088e+01,  9.98998235e-01,  1.70270825e+01])
Cost function after refinement: 7.771891010372871e-10
GonioParam(dist=0.7206860412873665, poni1=0.0348226304357366, poni2=0.004042897251336869, rot1=-3.357296798617952e-05, offset=-71.59990883821092, scale=0.9989982349992665, nrj=17.027082549190933)
maxdelta on: scale (5) 0.9989828627069963 --> 0.9989982349992665


array([ 7.20686041e-01,  3.48226304e-02,  4.04289725e-03, -3.35729680e-05,
       -7.15999088e+01,  9.98998235e-01,  1.70270825e+01])

In [58]:
gonio1 = complete_gonio(module_id=1)
gonio1.refine2()


Number of peaks previously found: 1183
Number of peaks found after re-scan: 1285
Cost function before refinement: 9.822658405373417e-10
[ 7.20682081e-01  3.36782425e-02  4.05801126e-03 -4.44860018e-05
 -7.71999788e+01  9.98967223e-01  1.70270825e+01]
     fun: 9.683803822381625e-10
     jac: array([ 1.05204414e-07, -1.40910812e-09, -2.64455137e-09,  1.49411766e-09,
       -3.43193224e-10,  4.36284460e-09,  1.50605559e-07])
 message: 'Optimization terminated successfully.'
    nfev: 37
     nit: 4
    njev: 4
  status: 0
 success: True
       x: array([ 7.20681597e-01,  3.36766788e-02,  4.05808287e-03, -4.45357208e-05,
       -7.71999788e+01,  9.98975895e-01,  1.70270825e+01])
Cost function after refinement: 9.683803822381625e-10
GonioParam(dist=0.7206815966774245, poni1=0.03367667883450645, poni2=0.004058082874451283, rot1=-4.4535720801764846e-05, offset=-77.19997881123506, scale=0.9989758952763078, nrj=17.027082549190933)
maxdelta on: scale (5) 0.9989672227193093 --> 0.998975895276307

array([ 7.20681597e-01,  3.36766788e-02,  4.05808287e-03, -4.45357208e-05,
       -7.71999788e+01,  9.98975895e-01,  1.70270825e+01])

In [59]:
gonio0 = complete_gonio(module_id=0)
gonio0.refine2()

Number of peaks previously found: 1203
Number of peaks found after re-scan: 1255
Cost function before refinement: 2.580092692155849e-06
[ 7.23096248e-01  3.20537111e-02  3.98485468e-03  1.09479202e-05
 -8.27999440e+01  9.99415047e-01  1.70270825e+01]
     fun: 2.5799877551962334e-06
     jac: array([-1.43881863e-07,  3.61314392e-08,  5.52834337e-07, -3.99174581e-07,
        1.01925934e-09, -1.14581923e-07, -1.57262758e-06])
 message: 'Optimization terminated successfully.'
    nfev: 37
     nit: 4
    njev: 4
  status: 0
 success: True
       x: array([ 7.23096999e-01,  3.20658547e-02,  3.98130190e-03,  1.35139061e-05,
       -8.27999438e+01,  9.99396229e-01,  1.70270825e+01])
Cost function after refinement: 2.5799877551962334e-06
GonioParam(dist=0.7230969991414224, poni1=0.03206585469442816, poni2=0.003981301898312368, rot1=1.351390607705048e-05, offset=-82.79994383949249, scale=0.9993962290572798, nrj=17.027082549190933)
maxdelta on: scale (5) 0.9994150470263444 --> 0.999396229057279

array([ 7.23096999e-01,  3.20658547e-02,  3.98130190e-03,  1.35139061e-05,
       -8.27999438e+01,  9.99396229e-01,  1.70270825e+01])

In [60]:
#Rescan module0 which looks much different:
gonio0.single_geometries.clear()
gonio0 = complete_gonio(module_id=0)
gonio0.refine2()

Number of peaks previously found: 0
Number of peaks found after re-scan: 1250
Cost function before refinement: 9.454138761636617e-07
[ 7.23096999e-01  3.20658547e-02  3.98130190e-03  1.35139061e-05
 -8.27999438e+01  9.99396229e-01  1.70270825e+01]
     fun: 9.315197221462237e-07
     jac: array([-1.20646249e-08, -1.14537357e-08,  1.03723337e-07, -7.48339701e-08,
        1.34221523e-10, -9.52891099e-09, -5.30292098e-07])
 message: 'Optimization terminated successfully.'
    nfev: 82
     nit: 9
    njev: 9
  status: 0
 success: True
       x: array([ 7.21973759e-01,  3.22003255e-02,  3.94381184e-03,  4.51413618e-05,
       -8.27999423e+01,  9.99140776e-01,  1.70270825e+01])
Cost function after refinement: 9.315197221462237e-07
GonioParam(dist=0.7219737589812981, poni1=0.03220032545666352, poni2=0.003943811844450548, rot1=4.5141361782464955e-05, offset=-82.79994226310926, scale=0.9991407764529461, nrj=17.027082549190933)
maxdelta on: dist (0) 0.7230969991414224 --> 0.7219737589812981


array([ 7.21973759e-01,  3.22003255e-02,  3.94381184e-03,  4.51413618e-05,
       -8.27999423e+01,  9.99140776e-01,  1.70270825e+01])

## Discard wronly assigned peaks

We have seen previously that some modules have a much higher residual error, while all have almost the same number of peaks recorded and fitted.

Some frames are contributing much more than all the other in those badly-fitted data. 
Let's spot them and re-assign them

In [61]:
#search for mis-assigned peaks in module #0
labels = []
errors = []

for lbl,sg in gonio0.single_geometries.items():
    labels.append(lbl)
    errors.append(sg.geometry_refinement.chi2())

s = numpy.argsort(errors)
for i in s[-10:]:
    print(labels[i], errors[i])
    
    

data_02_0455 6.313051349592421e-07
data_02_0464 6.523630272471416e-07
data_02_0465 6.891655641669719e-07
data_02_0457 7.370430556294668e-07
data_02_0456 7.376242032882277e-07
data_02_0460 7.45857621330519e-07
data_02_0466 7.67143438989774e-07
data_02_0461 7.935716411266278e-07
data_02_0462 7.971563845471846e-07
data_02_0480 0.001130766003993148


In [62]:
#remove wrongly assigned peak for frame 480
print(gonio0.single_geometries.pop("data_02_0480").control_points)
gonio0.refine2()
gonio0 = complete_gonio(module_id=0)
gonio0.refine2()

ControlPoints instance containing 4 group of point:
LaB6 Calibrant with 109 reflections at wavelength 7.281587849134994e-11
Containing 4 groups of points:
#psg ring 52: 1 points
#psh ring 53: 1 points
#psi ring 54: 1 points
#psj ring 55: 1 points
Cost function before refinement: 8.041228082228945e-09
[ 7.21973759e-01  3.22003255e-02  3.94381184e-03  4.51413618e-05
 -8.27999423e+01  9.99140776e-01  1.70270825e+01]
     fun: 9.214430277838577e-10
     jac: array([-3.08678332e-08, -3.82866689e-08, -1.82877549e-07,  1.31905635e-07,
       -8.44315805e-10, -1.61540448e-08,  1.38174042e-07])
 message: 'Optimization terminated successfully.'
    nfev: 82
     nit: 9
    njev: 9
  status: 0
 success: True
       x: array([ 7.20595797e-01,  3.22844186e-02,  4.04252687e-03, -2.05352660e-05,
       -8.27999410e+01,  9.98975164e-01,  1.70270825e+01])
Cost function after refinement: 9.214430277838577e-10
GonioParam(dist=0.720595797367119, poni1=0.03228441857336863, poni2=0.0040425268658778706, rot1

array([ 7.20595804e-01,  3.22848040e-02,  4.04257464e-03, -2.05697223e-05,
       -8.27999410e+01,  9.98975176e-01,  1.70270825e+01])

In [63]:
def search_outliers(module_id=None, name=None, threshold=1.2):
    "Search for wrongly assigned peaks"
    if name is None:
        name = ds_names[module_id]
    gonioref = goniometers[name]
    labels = []
    errors = []

    for lbl,sg in gonioref.single_geometries.items():
        labels.append(lbl)
        errors.append(sg.geometry_refinement.chi2())
    s = numpy.argsort(errors)
    last = errors[s[-1]]
    to_remove = []
    for i in s[-1::-1]:
        lbl = labels[i]
        current = errors[i]
        print(lbl , current, last, last/current)
        if threshold*current<last:
            break
        last=current
        to_remove.append(lbl)
    return to_remove


        
for lbl in search_outliers(8):
    gonio8.single_geometries.pop(lbl)
gonio8.refine2()
gonio8 = complete_gonio(module_id=8)
gonio8.refine2()

data_11_0495 1.4653542984471542e-06 1.4653542984471542e-06 1.0
data_11_0496 1.4360982661347142e-06 1.4653542984471542e-06 1.0203718874970744
data_11_0499 1.4166548534532985e-06 1.4360982661347142e-06 1.0137248763409235
data_11_0497 1.4084572221561037e-06 1.4166548534532985e-06 1.0058202912862668
data_11_0498 1.3833016059917487e-06 1.4084572221561037e-06 1.0181851998547489
data_11_0500 1.3709107012975464e-06 1.3833016059917487e-06 1.0090384477139718
data_11_0492 1.3295890859461896e-06 1.3709107012975464e-06 1.0310784856675854
data_11_0489 1.3287700089647162e-06 1.3295890859461896e-06 1.0006164174205825
data_11_0491 1.3216144771699174e-06 1.3287700089647162e-06 1.0054142353298985
data_11_0490 1.301822888645974e-06 1.3216144771699174e-06 1.015202980909737
data_11_0494 1.2865582016523121e-06 1.301822888645974e-06 1.0118647465571768
data_11_0493 1.2861450147452866e-06 1.2865582016523121e-06 1.0003212599685793
data_11_0483 1.2244483499124188e-06 1.2861450147452866e-06 1.0503873151017604
data

array([ 7.20813996e-01,  3.20850095e-02,  3.96169993e-03,  2.75882048e-05,
       -3.71999988e+01,  9.98991082e-01,  1.70270825e+01])

In [64]:
print(gonio7.chi2())
for lbl in search_outliers(7):
    gonio7.single_geometries.pop(lbl)
gonio7.refine2()
gonio7 = complete_gonio(module_id=7)
gonio7.refine2()

6.982178014396509e-10
data_10_0292 4.478307386585731e-06 4.478307386585731e-06 1.0
data_10_0291 4.441611877573707e-06 4.478307386585731e-06 1.0082617549717265
data_10_0288 4.42110785615644e-06 4.441611877573707e-06 1.0046377564367075
data_10_0285 4.41519072824658e-06 4.42110785615644e-06 1.0013401749265336
data_10_0290 4.411402144902852e-06 4.41519072824658e-06 1.0008588161358412
data_10_0289 4.40449960049467e-06 4.411402144902852e-06 1.001567157460386
data_10_0284 4.3863439763980785e-06 4.40449960049467e-06 1.004139124563482
data_10_0275 4.376867017048265e-06 4.3863439763980785e-06 1.0021652381287574
data_10_0286 4.374938577698873e-06 4.376867017048265e-06 1.0004407923254561
data_10_0283 4.360023596330053e-06 4.374938577698873e-06 1.0034208487727851
data_10_0281 4.358112634515273e-06 4.360023596330053e-06 1.000438483805959
data_10_0282 4.355265304875879e-06 4.358112634515273e-06 1.0006537672082128
data_10_0278 4.352327511323348e-06 4.355265304875879e-06 1.0006749936774948
data_10_0280

array([ 7.20709502e-01,  3.08662430e-02,  3.96677226e-03,  2.39315396e-05,
       -4.27999029e+01,  9.99006765e-01,  1.70270825e+01])

In [65]:
print(gonio0.chi2())
print(len(search_outliers(0)))
# for lbl in search_outliers(7):
#     gonio7.single_geometries.pop(lbl)
# gonio7.refine2()
# gonio7 = complete_gonio(module_id=7)
# gonio7.refine2()

9.382286239666452e-10
data_02_0462 7.971563845471846e-07 7.971563845471846e-07 1.0
data_02_0461 7.935716411266278e-07 7.971563845471846e-07 1.004517227222822
data_02_0466 7.67143438989774e-07 7.935716411266278e-07 1.034450144254191
data_02_0460 7.45857621330519e-07 7.67143438989774e-07 1.0285387144282092
data_02_0456 7.376242032882277e-07 7.45857621330519e-07 1.0111620768483298
data_02_0457 7.370430556294668e-07 7.376242032882277e-07 1.000788485359603
data_02_0465 6.891655641669719e-07 7.370430556294668e-07 1.0694716827883974
data_02_0464 6.523630272471416e-07 6.891655641669719e-07 1.056414197897037
data_02_0455 6.313051349592421e-07 6.523630272471416e-07 1.0333561238801883
data_02_0463 6.240813179531244e-07 6.313051349592421e-07 1.0115751213797115
data_02_0459 6.163901121421231e-07 6.240813179531244e-07 1.0124778215281103
data_02_0448 6.082027920092041e-07 6.163901121421231e-07 1.013461497119854
data_02_0454 5.868929875551012e-07 6.082027920092041e-07 1.036309523040778
data_02_0439 5.

## Overlay of the differents results

We are getting to an end. Here are the first actually integrated data

In [66]:
fig, ax = plt.subplots()
summed, counted, radial = None, None, None

for i in range(9):
    name = ds_names[i]
    ds = data[name]
    gonioref = goniometers[name]
    mg = gonioref.get_mg(position)
    mg.radial_range = (0, 95)
    images = [i.reshape(-1, 1) for i in ds]
    res_mg = mg.integrate1d(images, 50000)
    results[name] = res_mg    
    if summed is None:
        summed = res_mg.sum
        counted = res_mg.count
    else:
        summed += res_mg.sum
        counted += res_mg.count
    radial = res_mg.radial
    jupyter.plot1d(res_mg, label="%i %s"%(i, name), calibrant=LaB6, ax=ax )
    
ax.plot(radial, summed/counted, label="Merged")
ax.legend()
fig.show()    

/mntdirect/_scisoft/users/jupyter/jupy35/lib/python3.5/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/mntdirect/_scisoft/users/jupyter/jupy35/lib/python3.5/site-packages/matplotlib/__init__.py:932: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


area_pixel=0.05054585816712631 area_sum=0.05084240089824814, Error= -0.005866805745810646
area_pixel=0.05448433978731071 area_sum=0.05460194376540826, Error= -0.002158491385903477
area_pixel=0.05706893234299759 area_sum=0.057097754167008734, Error= -0.0005050352762500703
area_pixel=0.04575136894640419 area_sum=0.046103520693760155, Error= -0.007697075638731046
area_pixel=0.0503875602291064 area_sum=0.05041948354020665, Error= -0.0006335554044510445
area_pixel=0.050887046288177196 area_sum=0.05090400923674137, Error= -0.0003333451202514585
area_pixel=0.047764037615231736 area_sum=0.047787742717740125, Error= -0.0004962960355099896
area_pixel=0.054600585079026764 area_sum=0.05461392295287675, Error= -0.00024428078619822223
area_pixel=0.04398098576245957 area_sum=0.0451093409852144, Error= -0.025655523704017393
area_pixel=0.05413585962925449 area_sum=0.054261047973434326, Error= -0.0023124846458000116
area_pixel=0.05074179597564754 area_sum=0.051022305153632666, Error= -0.0055281681026771

/mntdirect/_scisoft/users/jupyter/jupy35/lib/python3.5/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in true_divide


FigureCanvasNbAgg()

## Multi-Gonio fit

Can we fit everything togeather ?
Just assume energy and scale parameter of the goniometer are the same for all modules and fit everything.

In [67]:
from scipy.optimize import minimize
class MultiGoniometer:
    def __init__(self, list_of_goniometers,
                param_name_split,
                param_name_common):
        self.nb_gonio = len(list_of_goniometers)
        self.goniometers = list_of_goniometers
        self.names_split = param_name_split
        self.names_common = param_name_common
        self.param = None
        
    def init_param(self):
        param = []
        for gonio in self.goniometers:
            param += list(gonio.param[:len(self.names_split)])
        param += list(gonio.param[len(self.names_split):])
        self.param = numpy.array(param)
    
    def residu2(self, param):
        "Actually performs the calulation of the average of the error squared"
        sumsquare = 0.0
        npt = 0
        for idx, gonio in enumerate(self.goniometers):
            gonio_param = numpy.concatenate((param[len(self.names_split)*idx:len(self.names_split)*(1+idx)],
                                             param[len(self.names_split)*len(self.goniometers):]))
            sumsquare += gonio.residu2(gonio_param)
        return sumsquare

    def chi2(self, param=None):
        """Calculate the average of the square of the error for a given parameter set
        """
        if param is not None:
            return self.residu2(param)
        else:
            if self.param is None:
                self.init_param()
            return self.residu2(self.param)
    def refine2(self, method="slsqp", **options):
        """Geometry refinement tool

        See https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.optimize.minimize.html

        :param method: name of the minimizer
        :param options: options for the minimizer
        """
        if method.lower() in ["simplex", "nelder-mead"]:
            method = "Nelder-Mead"

        former_error = self.chi2()
        print("Cost function before refinement: %s" % former_error)
        param = numpy.asarray(self.param, dtype=numpy.float64)
        print(param)
        res = minimize(self.residu2, param, method=method,
                       tol=1e-12,
                       options=options)
        print(res)
        newparam = res.x
        new_error = res.fun
        print("Cost function after refinement: %s" % new_error)

        if new_error < former_error:
            self.param = newparam
        return self.param
    
    def integrate(self, list_of_dataset, npt=50000, radial_range=(0,100)):
        summed = None
        counted = None
        param = self.param
        for idx, ds in enumerate(list_of_dataset):
            gonio = self.goniometers[idx]
            gonio_param = numpy.concatenate((param[len(self.names_split)*idx:len(self.names_split)*(1+idx)],
                                             param[len(self.names_split)*len(self.goniometers):]))
            print(gonio_param)
            gonio.param = gonio_param
            mg = gonio.get_mg(position)
            mg.radial_range = radial_range
            images = [i.reshape(-1, 1) for i in ds]
            res_mg = mg.integrate1d(images, 50000)
            if summed is None:
                summed = res_mg.sum
                counted = res_mg.count
            else:
                summed += res_mg.sum
                counted += res_mg.count
            radial = res_mg.radial
        res = Integrate1dResult(radial, summed/numpy.maximum(counted, 1e-10))
        res._set_unit(res_mg.unit)
        res._set_count(counted)
        res._set_sum(summed)
        return res

In [68]:
multigonio = MultiGoniometer([goniometers[ds_names[i]] for i in range(9)],
                 ["dist", "poni1", "poni2", "rot1", "offset"], 
                ["scale", "nrj"])


In [69]:
%time print(multigonio.chi2())
multigonio.param = numpy.array([ 7.20594053e-01,  3.22408604e-02,  4.05228023e-03, -2.75578440e-05,
       -8.27999414e+01,  7.20612302e-01,  3.36369797e-02,  4.02094516e-03,
       -1.74996556e-05, -7.71999791e+01,  7.20636130e-01,  3.47920978e-02,
        4.01341931e-03, -1.21330600e-05, -7.15999090e+01,  7.20757808e-01,
        3.33850817e-02,  3.95036100e-03,  3.46517345e-05, -6.57999267e+01,
        7.20813915e-01,  3.22167822e-02,  3.97128822e-03,  2.00055269e-05,
       -6.00000525e+01,  7.20881596e-01,  3.33801850e-02,  3.97760147e-03,
        1.47074593e-05, -5.43998157e+01,  7.21048510e-01,  3.22346939e-02,
        4.02104962e-03, -1.69519259e-05, -4.85998856e+01,  7.21074630e-01,
        3.08484557e-02,  4.09385968e-03, -6.91378973e-05, -4.27999030e+01,
        7.21154891e-01,  3.20619921e-02,  4.24950906e-03, -1.81328256e-04,
       -3.71999987e+01,  9.99038595e-01,  1.70266104e+01])
%time print(multigonio.chi2())


8.603112596188815e-09
CPU times: user 9.77 s, sys: 12 ms, total: 9.78 s
Wall time: 884 ms
6.1395983620232986e-09
CPU times: user 8.64 s, sys: 4 ms, total: 8.65 s
Wall time: 770 ms


In [70]:
%time multigonio.refine2()

Cost function before refinement: 6.1395983620232986e-09
[ 7.20594053e-01  3.22408604e-02  4.05228023e-03 -2.75578440e-05
 -8.27999414e+01  7.20612302e-01  3.36369797e-02  4.02094516e-03
 -1.74996556e-05 -7.71999791e+01  7.20636130e-01  3.47920978e-02
  4.01341931e-03 -1.21330600e-05 -7.15999090e+01  7.20757808e-01
  3.33850817e-02  3.95036100e-03  3.46517345e-05 -6.57999267e+01
  7.20813915e-01  3.22167822e-02  3.97128822e-03  2.00055269e-05
 -6.00000525e+01  7.20881596e-01  3.33801850e-02  3.97760147e-03
  1.47074593e-05 -5.43998157e+01  7.21048510e-01  3.22346939e-02
  4.02104962e-03 -1.69519259e-05 -4.85998856e+01  7.21074630e-01
  3.08484557e-02  4.09385968e-03 -6.91378973e-05 -4.27999030e+01
  7.21154891e-01  3.20619921e-02  4.24950906e-03 -1.81328256e-04
 -3.71999987e+01  9.99038595e-01  1.70266104e+01]
     fun: 6.1395983620232986e-09
     jac: array([ 5.90883473e-08, -1.08561671e-07, -2.52646911e-08,  1.79747064e-08,
       -1.72926662e-09,  1.79298213e-08,  9.69570013e-09,  4.

array([ 7.20594053e-01,  3.22408604e-02,  4.05228023e-03, -2.75578440e-05,
       -8.27999414e+01,  7.20612302e-01,  3.36369797e-02,  4.02094516e-03,
       -1.74996556e-05, -7.71999791e+01,  7.20636130e-01,  3.47920978e-02,
        4.01341931e-03, -1.21330600e-05, -7.15999090e+01,  7.20757808e-01,
        3.33850817e-02,  3.95036100e-03,  3.46517345e-05, -6.57999267e+01,
        7.20813915e-01,  3.22167822e-02,  3.97128822e-03,  2.00055269e-05,
       -6.00000525e+01,  7.20881596e-01,  3.33801850e-02,  3.97760147e-03,
        1.47074593e-05, -5.43998157e+01,  7.21048510e-01,  3.22346939e-02,
        4.02104962e-03, -1.69519259e-05, -4.85998856e+01,  7.21074630e-01,
        3.08484557e-02,  4.09385968e-03, -6.91378973e-05, -4.27999030e+01,
        7.21154891e-01,  3.20619921e-02,  4.24950906e-03, -1.81328256e-04,
       -3.71999987e+01,  9.99038595e-01,  1.70266104e+01])

In [71]:
LaB6_new = get_calibrant("LaB6")
LaB6_new.wavelength = 1e-10*hc/multigonio.param[-1]
print(LaB6,"\n", LaB6_new)

LaB6 Calibrant with 109 reflections at wavelength 7.281789761742598e-11 
 LaB6 Calibrant at wavelength 7.281789768115395e-11


In [72]:
%time res = multigonio.integrate([data[ds_names[i]] for i in range(9)])

[ 7.20594053e-01  3.22408604e-02  4.05228023e-03 -2.75578440e-05
 -8.27999414e+01  9.99038595e-01  1.70266104e+01]
area_pixel=0.053413862261281864 area_sum=0.055214882933248166, Error= -0.033718225863472326
area_pixel=0.045333067243413794 area_sum=0.04986384765291535, Error= -0.09994427213966664
area_pixel=0.03918749031126367 area_sum=0.04563241935519968, Error= -0.16446393970995232
area_pixel=0.04641997394066166 area_sum=0.05056017758649528, Error= -0.0891901329183427
area_pixel=0.04963019502342725 area_sum=0.052784706110374724, Error= -0.06356032019335069
area_pixel=0.041808779534353135 area_sum=0.04729034242949584, Error= -0.13111033032281305
area_pixel=0.053529818242976646 area_sum=0.05553764984245543, Error= -0.03750865714441734
area_pixel=0.05746249488342414 area_sum=0.05828939512131753, Error= -0.014390259935127214
area_pixel=0.03804626259786659 area_sum=0.04468957516491201, Error= -0.17461143653615904
area_pixel=0.0579645518625469 area_sum=0.05860457337617547, Error= -0.0110416

[ 7.21154891e-01  3.20619921e-02  4.24950906e-03 -1.81328256e-04
 -3.71999987e+01  9.99038595e-01  1.70266104e+01]
area_pixel=0.3162483597934198 area_sum=0.3174253016954389, Error= -0.003721574723068656
area_pixel=0.24311057254745805 area_sum=0.2600497496702889, Error= -0.0696768427030219
area_pixel=0.16860365236765773 area_sum=0.21967653369346593, Error= -0.3029168147226045
area_pixel=0.09285566528839695 area_sum=0.19579635166280646, Error= -1.108609647614821
area_pixel=0.01576774945500148 area_sum=0.18814793828945114, Error= -10.932453570903947
area_pixel=0.06752863879084003 area_sum=0.19124575955043382, Error= -1.8320689262342347
area_pixel=0.14373310781029858 area_sum=0.20978466196882295, Error= -0.4595430737203592
area_pixel=0.2186034780352415 area_sum=0.24470325423963207, Error= -0.11939323399137768
area_pixel=0.29216108377022465 area_sum=0.2964675523040342, Error= -0.014740048463115846
area_pixel=0.2878799981328086 area_sum=0.29298295310643363, Error= -0.01772597959817564
area_p

CPU times: user 1min 36s, sys: 72 ms, total: 1min 36s
Wall time: 8.76 s


In [73]:
ax = jupyter.plot1d(res, calibrant=LaB6_new)
ax.figure.show()

/mntdirect/_scisoft/users/jupyter/jupy35/lib/python3.5/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/mntdirect/_scisoft/users/jupyter/jupy35/lib/python3.5/site-packages/matplotlib/__init__.py:932: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


FigureCanvasNbAgg()

In [74]:
fig, ax = plt.subplots()
ax.plot(*calc_fwhm(res, LaB6_new, 10, 95), "o", label="FWHM")
ax.plot(*calc_peak_error(res, LaB6_new, 10, 95), "o", label="error")
ax.set_title("Peak shape & error as function of the angle")
ax.set_xlabel(res.unit.label)
ax.legend()
fig.show()

/mntdirect/_scisoft/users/jupyter/jupy35/lib/python3.5/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/mntdirect/_scisoft/users/jupyter/jupy35/lib/python3.5/site-packages/matplotlib/__init__.py:932: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


28457
68 60


FigureCanvasNbAgg()

In [75]:
print("total run time: ", time.time()-start_time)

total run time:  937.571061372757


## Conclusion
The calibration works and the FWHM of every single peak is pretty small: 0.02°. 
The geometry has been refined with the wavelength: 
The goniometer scale parameter refines to 0.999 instead of 1 and the wavelength is fitted with a change at the 5th digit which is pretty precise.